In [1]:
import pandas as pd
import numpy as np
import os

Fucntions of this notebook:
* Read wealth process data ("ticker_wealth.csv" in directory ".../Output", generated by 5_strategy_backtest)
* Calcualte and print Sharpe Ratio, Maximum Drawdown and Calmar Ratio
* Generate holding period and P&L for each trade, and output the results as one .csv file (with name "ticker_trades.csv" and stored in directory ".../Output")

### Directory Setting 

In [2]:
dir_working = "/Users/user/Desktop/E4733 AT/Project/Coding Environment/Codes"
os.chdir(dir_working)
dir_data = "../Output"
dir_output = "../Output"
ticker = "aapl"
ticker_list = ['aapl', 'amzn', 'nvda', 'amd', 'msft', 'fb', 'nflx', \
               'goog', 'intc', 'pypl']

### 1. Evaluation Function

In [3]:
# option: ratio, trades, both
def envaluation(ticker = 'aapl', option = 'both'):
    data = pd.read_csv(dir_data + '/' + ticker + "_wealth.csv")
    data.time = pd.to_datetime(data.time)
    data = data.set_index('time', drop = True)
    signal = data['signal'].values 
    wealth = data['wealth'].values
    data['trade_dir'] = data['signal'].shift(1)
    ## ratio calculation 
    if option != 'trades':
        data['wealth_diff'] = data.wealth - data.wealth.shift(1)
        # return
        hpr = wealth[len(wealth)-1]/wealth[0] - 1
        # sharpe ratio
        r = data.wealth_diff.mean()/wealth[0]
        r_sd = data.wealth_diff.std()/wealth[0]
        n_min = 10*6.5*60
        sharpe_ratio = r/r_sd * np.sqrt(n_min)
        # maximum drawdown
        max_inner = 0
        max_outter = 0
        p_inner = 0
        p_outter = 0
        for i in range(1,len(wealth)):
            for j in range(0,i):
                if wealth[j] - wealth[i] > max_inner:
                    max_inner = wealth[j] - wealth[i]
                    t_p_inner = j
            if max_inner > max_outter:
                max_outter = max_inner
                p_outter = i
                p_inner = t_p_inner
            t_p_inner = 0
            max_inner = 0
        max_dd = wealth[p_inner] - wealth[p_outter]
        max_dd_p = (p_outter - p_inner)
        # calmar ratio 
        if max_dd == 0:
            calmar_ratio = float('inf')
        else:
            calmar_ratio = (wealth[len(wealth)-1]-wealth[0])/max_dd
    ## trade summary
    if option != 'ratio':
        trade_end = np.zeros(len(signal))
        hp = np.zeros(len(signal))
        trade_pnl = np.zeros(len(signal))
        # when i == 0
        trade_end[0] = np.nan
        hp_t = 0
        for i in range(1,len(signal)-1):
            if signal[i] != signal[i-1]:
                # trade_end:
                if signal[i-1] != 0:
                    trade_end[i] = 1
                    hp_t = hp_t + 1
                    trade_pnl[i] = wealth[i] - wealth[i-hp_t]
                    hp[i] = hp_t
                    hp_t = 0
                else: # a new trade 
                    trade_end[i] = np.nan
                    hp_t = 0
            else:
                trade_end[i] = np.nan
                if signal[i-1] != 0:
                    hp_t += 1
                    hp[i] = hp_t
                else:
                    trade_end[i] = np.nan
        data['trade_end'] = trade_end
        data['hp'] = hp
        data['trade_pnl'] = trade_pnl
        data = data.dropna()
        data = data.drop(['open', 'high', 'low', 'close', 'volume','splits', \
                          'earnings', 'dividend', 'average_price', 'return', \
                          'direction', 'signal_logistic_full', \
                          'logistic_prob_full', 'signal_logistic', \
                          'logistic_prob', 'wealth'],axis=1)
        data.to_csv(dir_output + '/' + ticker + '_trades.csv')
    ## return
    if option != 'trades':
        return [hpr,sharpe_ratio, max_dd/wealth[0], max_dd_p, calmar_ratio]

### 2. Execution

In [4]:
for i in range(len(ticker_list)):
    # trades summary for 0 tc
    dir_data = "../Output"
    envaluation(ticker_list[i], option = 'trades')
    # ratios for 3bp tc
    dir_data = "../Output"
    ratio_list = envaluation(ticker_list[i], option = 'ratio')
    print(ticker_list[i])
    print('Holding Period Return: ', ratio_list[0])
    print('Sharpe Ratio: ', ratio_list[1])
    print('Maximum Drawdown: ', ratio_list[2])
    print('Drawdown Period Length: ', ratio_list[3], ' min')
    print('Calmar Ratio: ', ratio_list[4], '\n')

aapl
Holding Period Return:  0.6548351757492454
Sharpe Ratio:  7.705290221764635
Maximum Drawdown:  0.07669285918217134
Drawdown Period Length:  155  min
Calmar Ratio:  8.538411303636389 

amzn
Holding Period Return:  2.474882306361947
Sharpe Ratio:  12.025404920196095
Maximum Drawdown:  0.21772583343963242
Drawdown Period Length:  152  min
Calmar Ratio:  11.36696673639393 

nvda
Holding Period Return:  3.409908854349718
Sharpe Ratio:  14.337880243953714
Maximum Drawdown:  0.18566333940347748
Drawdown Period Length:  113  min
Calmar Ratio:  18.36608597747677 

amd
Holding Period Return:  6.360006595348604
Sharpe Ratio:  15.775557172146973
Maximum Drawdown:  0.1772439800302782
Drawdown Period Length:  117  min
Calmar Ratio:  35.88277917400714 

msft
Holding Period Return:  1.1510508830335224
Sharpe Ratio:  10.138477082907713
Maximum Drawdown:  0.0702797426512243
Drawdown Period Length:  101  min
Calmar Ratio:  16.378131729164362 

fb
Holding Period Return:  1.663803916668841
Sharpe Rati